<a href="https://colab.research.google.com/github/MuhammadUsman-Khan/Fake-News-Prediction/blob/main/Project_4_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# printing the stopwords in English
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Prepocessing

In [ ]:
# loading the dataset into the Pandas dataframe
news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='warn', engine='python')

<ipython-input-14-d8a9b354ea9e>:2: ParserWarning: Skipping line 13535: field larger than field limit (131072)

  news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='warn', engine='python')
<ipython-input-14-d8a9b354ea9e>:2: ParserWarning: Skipping line 19783: field larger than field limit (131072)

  news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='warn', engine='python')
<ipython-input-14-d8a9b354ea9e>:2: ParserWarning: Skipping line 13541: Expected 5 fields in line 13541, saw 6

  news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='warn', engine='python')
<ipython-input-14-d8a9b354ea9e>:2: ParserWarning: Skipping line 19782: Expected 5 fields in line 19782, saw 7

  news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='warn', engine='python')
<ipython-input-14-d8a9b354ea9e>:2: ParserWarning: Skipping line 19783: Expected 5 fields in line 19783, saw 10

  news_dataset = pd.read_csv('/content/train.csv', on_bad_lines='warn', engine='python')

In [ ]:
news_dataset.shape

(20822, 5)

In [ ]:
# print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# counting the number of missing values in the dataframe
news_dataset.isnull().sum()

,0
id,0
title,565
author,1975
text,59
label,22


In [ ]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [ ]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20817    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20818    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20819    Michael J. de la Merced and Rachel Abrams Macy...
20820    Alex Ansary NATO, Russia To Hold Parallel Exer...
20821              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20822, dtype: object


In [ ]:
# seperating the data and the label
X = news_dataset.drop(columns='label', axis=1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20817  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20818  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20819  20797  Macy’s Is Said to Receive Takeover Approach by...   
20820  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20821  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming

Stemming is the process of reducing a word into its root word


Example: actress, actor, acting -----> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20817    jerom hudson rapper trump poster child white s...
20818    benjamin hoffman n f l playoff schedul matchup...
20819    michael j de la merc rachel abram maci said re...
20820    alex ansari nato russia hold parallel exercis ...
20821                            david swanson keep f aliv
Name: content, Length: 20822, dtype: object


In [ ]:
# Seperating the labels and the data
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


In [ ]:
print(Y)

['1' '0' '1' ... '0' '1' '1']


In [ ]:
Y.shape

(20822,)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 210778 stored elements and shape (20822, 17136)>
  Coords	Values
  (0, 267)	0.27014778053672167
  (0, 2484)	0.3676990592743697
  (0, 2960)	0.24689141014750257
  (0, 3602)	0.35994096489813754
  (0, 3794)	0.2705797813233836
  (0, 4975)	0.2333632865672138
  (0, 7008)	0.21878792690522433
  (0, 7695)	0.24789859828097843
  (0, 8633)	0.29217179802324905
  (0, 8912)	0.36364344795259246
  (0, 13478)	0.2560149172431211
  (0, 15695)	0.2848972510323515
  (1, 1497)	0.2939921887587299
  (1, 1894)	0.15524631265972175
  (1, 2224)	0.3827200188088641
  (1, 2814)	0.19096625123288233
  (1, 3570)	0.2637458518155929
  (1, 5506)	0.7143145691090199
  (1, 6819)	0.1904866118084521
  (1, 16807)	0.30071934786077886
  (2, 2944)	0.3180050294894952
  (2, 3104)	0.4609666076640373
  (2, 5392)	0.3866575731579265
  (2, 5971)	0.3474726097377811
  (2, 9623)	0.4935010343775579
  :	:
  (20819, 3645)	0.21156327884082238
  (20819, 7045)	0.21799779675029382
  (20819

Splitting the Data into training and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

Training the Model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("Accuracy score of the Training Data : ", training_data_accuracy)

Accuracy score of the Training Data :  0.9923755778351444


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print("Accuracy score of the Test Data : ", test_data_accuracy)

Accuracy score of the Test Data :  0.9817527010804322


Building a Predictive System

In [ ]:
X_new = X_test[10]

# Convert the new input string to a list
X_new_list = [X_new]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]=='0'):
  print('The news is Real')

else:
  print('The news is Fake')

['1']
The news is Fake


In [ ]:
print(Y_test[10])

1
